In [ ]:
#%pip uninstall importlib_metadata -y
#%pip install importlib_metadata --force-reinstall

In [ ]:
# %pip install keras

In [ ]:
#%env SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True

In [ ]:
#%pip install --force-reinstall -v "wfdb==1.3.9"

In [ ]:
#%pip install --force-reinstall -v "wfdb==4.1.2"

In [ ]:
# %pip install wfdb
# %pip install tqdm

In [2]:
%pip install pandas
%pip install keras
%pip install wfdb
%pip install tqdm
%pip install scikit-learn

  Using cached pandas-2.1.3-cp39-cp39-win_amd64.whl.metadata (18 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
Using cached pandas-2.1.3-cp39-cp39-win_amd64.whl (10.8 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached keras-3.0.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached rich-13.7.0-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.7-py3-none-any.whl (5.8 kB)
  Using cached dm_tree-0.1.8-cp39-cp39-win_amd64.whl (101 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
Using cached keras-3.0.0-py3-none-any.whl (997 kB)
Using cached rich-13.7.0-py3-none-any.whl (240 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires clang~=5.0, which is not installed.
tensorflow 2.6.0 requires absl-py~=0.10, but you have absl-py 1.4.0 which is incompatible.
tensorflow 2.6.0 requires flatbuffers~=1.12, but you have flatbuffers 20210226132247 which is incompatible.


  Using cached wfdb-4.1.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl (1.0 MB)
  Using cached matplotlib-3.8.2-cp39-cp39-win_amd64.whl.metadata (5.9 kB)
  Using cached contourpy-1.2.0-cp39-cp39-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.46.0-cp39-cp39-win_amd64.whl.metadata (159 kB)
  Using cached kiwisolver-1.4.5-cp39-cp39-win_amd64.whl.metadata (6.5 kB)
  Using cached Pillow-10.1.0-cp39-cp39-win_amd64.whl.metadata (9.6 kB)
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
  Using cached importlib_resources-6.1.1-py3-none-any.whl.metadata (4.1 kB)
Using cached wfdb-4.1.2-py3-none-any.whl (159 kB)
Using cached matplotlib-3.8.2-cp39-cp39-win_amd64.whl (7.6 MB)
Using cached contourpy-1.2.0-cp39-cp39-win_amd64.whl (181 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.46.0-cp39-cp39-win_amd64.whl (2.2 MB)
Using ca

In [12]:
import glob
import os
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import wfdb
from tqdm import tqdm

C:\Users\School\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:585: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  np.object,


AttributeError: module 'numpy' has no attribute 'object'.
`np.object` was a deprecated alias for the builtin `object`. To avoid this error in existing code, use `object` by itself. Doing this will not modify any behavior and is safe. 
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [13]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Assuming you want to use the first GPU


In [14]:

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


NameError: name 'tf' is not defined

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

1. Data Collection:
·Datasets Used:
·CHF-RR Dataset: Congestive heart failure RR interval database. It comprised patients with heart failure. This dataset contained records of 29 patients from CHF201 to CHF229.
·NSR-RR Dataset: Normal sinus rhythm RR interval database. This dataset had 54 normal sinus rhythm recordings ranging in age from 28 to 76. Records from NSR001 through NSR054 of 54 patients were available.
·Both datasets were provided by PhysioBank.
2. Data Preprocessing:
·Conversion: Datasets were converted to CSV files or directly imported into Python.
·Library Used: The wfdblibrary was employed to read, write, and process WFDB signals and annotations. The library can be accessed at wfdb library.
·Grouping Approach: Among various grouping methods for the TSC algorithm, intervals were chosen over whole series. Data was normalized and divided into intervals of 5sec, 20sec, 40sec, 60sec, and 80sec.
·Data Formatting: Data was formatted into univariate UCR time series.
3. Data Augmentation:
·Data Augmentation Strategy: To combat issues of underfitting and overfitting, data augmentation was mentioned as a strategy. However, specific augmentation techniques were not detailed in the provided information.

# Downloading Data

In [ ]:
dbs = wfdb.get_dbs()
display(dbs)

In [ ]:
def download(database):
    cwd = os.getcwd()
    dl_dir = os.path.join(cwd, database)
    wfdb.dl_database(database, dl_dir=dl_dir)
    display(os.listdir(dl_dir))

In [ ]:
# download('nsrdb')

In [ ]:
# download('chfdb')

# Converting Data

In [ ]:
def wfdb_to_dataframe(record):
    #df = record.to_dataframe()
    df = pd.DataFrame(record.p_signal, columns=record.sig_name)
    return df
 
def load_record(uri):
    record = wfdb.rdrecord(uri)
    return wfdb_to_dataframe(record)

In [ ]:
a = load_record('nsrdb/16265')
plt.plot(a['ECG1'][:212])
plt.show()

In [ ]:
plt.plot(a['ECG2'][:212])

In [ ]:
load_record('chfdb/chf03')

In [ ]:
nsrdb_list = glob.glob("nsrdb/*.hea")
len(nsrdb_list)

In [ ]:
chfdb_list = glob.glob("chfdb/*.hea")
len(chfdb_list)

In [ ]:
def create_db(file_list, record_class, clip_duration=60, sampling_rate=250):
    clip_len = sampling_rate * clip_duration
    num_channels = len(load_record(file_list[0].removesuffix('.hea')).columns)
    total_clips = sum(len(load_record(file_path.removesuffix('.hea'))) // clip_len for file_path in file_list)

    # Preallocate a NumPy array for all clips
    all_clips = np.empty((total_clips, clip_len * num_channels + 1))  # +1 for the class label

    clip_index = 0
    for file_path in file_list:
        record_df = load_record(file_path.removesuffix('.hea'))
        num_clips = len(record_df) // clip_len

        for i in range(num_clips):
            clip = record_df.iloc[i * clip_len : (i + 1) * clip_len].values.flatten()
            all_clips[clip_index, :-1] = clip  # Fill all but last column with clip data
            all_clips[clip_index, -1] = record_class  # Last column is the class label
            clip_index += 1

    # Convert the NumPy array directly to a DataFrame
    column_names = [f't{i}' for i in range(clip_len * num_channels)] + ['class']
    df_clips = pd.DataFrame(all_clips, columns=column_names)

    return df_clips


In [ ]:
nsrdb_df = create_db(nsrdb_list, 0)
nsrdb_df.shape

In [ ]:
chfdb_df = create_db(chfdb_list, 1)
chfdb_df.shape

In [ ]:
full_df = pd.concat([nsrdb_df, chfdb_df],axis=0, ignore_index=True, sort=False)
full_df.head()

# Split Data

In [ ]:
y_df = full_df.pop("class")
x_df = full_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.30, random_state=42)

In [ ]:
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size=0.20, random_state=42)

In [ ]:
print("Training Data:", X_train.shape)
print("Validation Data:", X_validate.shape)
print("Testing Data:", X_test.shape)

# Model

In [ ]:
input_shape = (X_train.shape[1], 1, )

The first convolutional layer will consist of 20 filters, followed by a max-pooling layer of length 2.
 
The second convolutional layer will have 80 filters with a max-pooling layer of length 4.
 
There will be 32 neurons used in the fully connected convolution layers to feed into the softmax classification layer.

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv1D(20, 5),
        tf.keras.layers.MaxPooling1D(2),
        tf.keras.layers.Conv1D(80, 5),
        tf.keras.layers.MaxPooling1D(4),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Dense(1, activation='sigmoid'),  # Sigmoid for binary classification
    ]
)


In [ ]:
model.summary()

# Model Training

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
batch_size = 32
epochs = 30

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

# Model Evaluation

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])